# Homework HW04 Solution

In [1]:
import pandas as pd
import numpy as np

The Employees, Territory, Customers, and Orders tables are the same as those we used in class.

In [2]:
Employees = pd.read_excel('/home/data/AdventureWorks/Employees.xls')
Territory = pd.read_excel('/home/data/AdventureWorks/SalesTerritory.xls')
Customers = pd.read_excel('/home/data/AdventureWorks/Customers.xls')
Orders = pd.read_excel('/home/data/AdventureWorks/ItemsOrdered.xls')

## Problem 1
Write a function called `get_manager` that takes as its one argument the Pandas DataFrame "Employees" and returns a DataFrame containing list of all employees (EmployeeID, first name, middle name, last name), and their manager's first and last name. The columns in the output DataFrame should be: EmployeeID, FirstName, MiddleName, LastName, ManagerFirstName, ManagerLastName.



In [11]:
def get_manager(df):
    
    # step 1: create two DFs for mappings of employeeID -> Manager ID and EmployeeID -> Employee Name
    # note that managers are also employees and hence manager IDs are also employee IDs
    employee_manager_df = df[["EmployeeID","FirstName","MiddleName","LastName","ManagerID"]]
    employee_id_name_df = df[["EmployeeID","FirstName","LastName"]]
    
    # Step 2: merge the two but make sure which name is for whom
    out = pd.merge(
        employee_manager_df
        , employee_id_name_df
        , left_on = 'ManagerID'
        , right_on = 'EmployeeID'
        , how = 'inner'  # i'm excluding those without managers but you can argue about that
        , suffixes = ('_Employee', '_Manager')  # optional but more explicit and easier to check for error 
    )
    
    # step 3: cleaning
    
    out = out.drop(['ManagerID', 'EmployeeID_Manager'], axis=1)
    out.columns = ["EmployeeID","FirstName","MiddleName","LastName","ManagerFirstName","ManagerLastName"]
    
    return out

In [12]:
df1 = get_manager(Employees)
print("Shape of resulting table: ", df1.shape)
print("Columns: ", ', '.join(df1.columns))
df1.head()

Shape of resulting table:  (290, 6)
Columns:  EmployeeID, FirstName, MiddleName, LastName, ManagerFirstName, ManagerLastName


,EmployeeID,FirstName,MiddleName,LastName,ManagerFirstName,ManagerLastName
0,259,Ben,T,Miller,Sheela,Word
1,255,Gordon,L,Hee,Sheela,Word
2,124,Kim,T,Ralls,Sheela,Word
3,257,Eric,S,Kurjan,Sheela,Word
4,126,Jimmy,T,Bischoff,Sheela,Word



    Shape of resulting table:  (291, 6)
    Columns:  EmployeeID, FirstName, MiddleName, LastName, ManagerFirstName, ManagerLastName

&nbsp;  | EmployeeID | FirstName |MiddleName | LastName | ManagerFirstName | ManagerLastName
---------|-----------|-----------|-----------|----------|------------------|----------------
0	| 259	| Ben	| T	| Miller	|Sheela	| Word
1	| 278	| Garrett	| R	| Vargas	|Stephen	| Jiang
2	| 204	| Gabe	| B	| Mares	| Peter	| Krebs
3	| 78	| Reuben	| H	| D'sa	| Peter	| Krebs
4	| 255	| Gordon	| L	| Hee	| Sheela	| Word


## Problem 2
Write a functon called `get_spend_by_order` that takes as its two arguments the Pandas DataFrames "Orders" and "Customers", and returns a DataFrame with the following columns: "FirstName", "LastName", "Item", "TotalSpent", listing all cutomer names, their purchased items, and the total amount spend on that item (remember that the "Price" listed in "Orders" is the _price per item_).



In [13]:
def get_spend_by_order(order_df, cust_df):
    
    # step 1: pick columns to be used
    cust_df_columns = ["CustomerID","FirstName","LastName"]
    order_df_columns = ["CustomerID","Item","Quantity","Price"]
    
    # Step 2: Merge two table
    order_cust = pd.merge(
        cust_df[cust_df_columns]
        , order_df[order_df_columns]
        , left_on = 'CustomerID'
        , right_on = 'CustomerID'
        , how = 'left'
    )
    
    # Step 3 : Create column total spent
    order_cust["TotalSpent"] = order_cust["Quantity"] * order_cust["Price"]

    # Step 4: Groupby and sum
    total_spent_on_item = order_cust.groupby(by = ["FirstName","LastName","Item"])["TotalSpent"].sum()

    # Step_4: Reset_index
    result = total_spent_on_item.reset_index()
    return result


In [14]:
df2 = get_spend_by_order(Orders, Customers)
print("Shape of resulting table: ", df2.shape)
print("Columns: ", ', '.join(df2.columns))
df2.head()

Shape of resulting table:  (32, 4)
Columns:  FirstName, LastName, Item, TotalSpent


,FirstName,LastName,Item,TotalSpent
0,Anthony,Sanchez,Umbrella,4.5
1,Conrad,Giles,Ski Poles,51.0
2,Conrad,Giles,Tent,88.0
3,Donald,Davids,Lawnchair,128.0
4,Elroy,Keller,Inflatable Mattress,38.0


    Shape of resulting table:  (32, 4)
    Columns:  FirstName, LastName, Item, TotalSpent
    
&nbsp;  |FirstName | LastName | Item | TotalSpent
-----|----------|----------|------|-----------
0 |	Anthony	| Sanchez	| Umbrella	| 4.5
1 |	Conrad	| Giles	| Ski Poles	| 25.5
2 |	Conrad	| Giles	| Tent	| 88.0
3 |	Donald	| Davids	| Lawnchair	| 32.0
4 |	Elroy	| Keller	| Inflatable Mattress	| 38.0

## Problem 3
Write a function called `get_order_location` that takes three arguments: "Orders", "Customers", and "Territory", and returns a DataFrame containing the following columns:  "CustomerID", "Name", and "TotalItems", that gives, for each order, the  CustomerID, the name of the territory where the order was placed, and the total number of items ordered (yes, 2 ski poles counts as 2 items). 



In [18]:
def get_order_location(df_order, df_cust, df_terri):
    
    step1_df = pd.merge(df_order, 
                       df_cust,
                       how = 'left',
                       left_on = 'CustomerID',
                       right_on = 'CustomerID')
    
    step2_df = pd.merge(step1_df, 
                       df_terri,
                       how = 'left',
                       left_on = 'SalesTerritoryID',
                       right_on = 'TerritoryID')

    step3_df = step2_df.groupby(by = ['CustomerID','Name']).Quantity.sum().reset_index()
    out = step3_df.rename(columns={"Quantity":"TotalItems"})
    return out

In [19]:
df3 = get_order_location(Orders, Customers, Territory)
print("Shape of resulting table: ", df3.shape)
print("Columns: ", ', '.join(df3.columns))
df3.head()

Shape of resulting table:  (11, 3)
Columns:  CustomerID, Name, TotalItems


,CustomerID,Name,TotalItems
0,10101,Northwest,12
1,10298,Southwest,6
2,10299,Northwest,2
3,10315,Central,1
4,10330,Northwest,6


    Shape of resulting table:  (11, 3)
    Columns:  CustomerID, Name, TotalItems
    

&nbsp;| CustomerID | Name | TotalItems
-----|-----------|------|-----------
0	| 10315	| Central	| 1
1	| 10438	| Central	| 3
2	| 10439	| Central	| 2
3	| 10101	| Northwest	| 6
4	| 10299	| Northwest	| 2

## Problem 4
Write a function called `employee_info` that takes one argument: "Employees", and returns a DataFrame containing the following columns: JobTitle, NumberOfEmployees, and MeanVacationHours, containing all job titles, the number of employees with that job title, and the mean number of vacation days for employees with that job title. 

In [30]:
def employee_info(df_emp):
    jobtitle_vacations = df_emp.groupby(by = ["JobTitle"]).VacationHours.aggregate(["count","mean"]).reset_index()

    new_columns = {"count":"NumberOfEmployees","mean":"MeanVacationHours"}
    out = jobtitle_vacations.rename(columns = new_columns)
    return out

In [31]:
df4 = employee_info(Employees)
print("Shape of resulting table: ", df4.shape)
print("Columns: ", ', '.join(df4.columns))
df4.head()

Shape of resulting table:  (68, 3)
Columns:  JobTitle, NumberOfEmployees, MeanVacationHours


,JobTitle,NumberOfEmployees,MeanVacationHours
0,Accountant,2,58.5
1,Accounts Manager,1,57.0
2,Accounts Payable Specialist,2,63.5
3,Accounts Receivable Specialist,3,61.0
4,Application Specialist,4,72.5


    Shape of resulting table:  (68, 3)
    Columns:  JobTitle, NumberOfEmployees, MeanVacationHours

&nbsp;  | JobTitle | NumberOfEmployees | MeanVacationHours
--- |----------|-------------------|------
0 |	Accountant	|2	|58.5
1 |	Accounts Manager	|1	|57.0
2 |	Accounts Payable Specialist	|2	|63.5
3 |	Accounts Receivable Specialist	|3	|61.0
4 |	Application Specialist	|4	|72.5